This is week 2 for the final assignment of the capstone project.
Here we are comparing the two cities of Rajasthan state of India : Jaipur and Udaipur.
Both cities have historic places, tourist places, cafes and restaurants and markets.
It can be a good project to compare the two cities and check the similarity of the two cities.
The comparison can help an investor to figure out what business is profitable in which city and also can help tourists
to decide which city to visit first.
We fetch the data from foursquare API for both the cities. Then we plot the places on interest of both the cities on map
and make a detailed study.

In [1]:
!pip install folium

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011


In [6]:
from geopy.geocoders import Nominatim 
import folium
import requests
import pandas as pd
import numpy as np

In [13]:
# Foursquare API configuration
CLIENT_ID = '44TLKI2RQKYUZGS0PUP2EG3YD1U2FQU1DJPMG4W4SEVSXR3V' 
CLIENT_SECRET = 'EEPPCM3JRVRDSUFJ0QW2C3UYRQIIA15JB5OKTHFBZH4LKBEK'
VERSION = '20200104'
LIMIT = 1000
radius = 10000

In [4]:
address = 'Jaipur, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
jaipur_latitude = location.latitude
jaipur_longitude = location.longitude
print('the coordinates of jaipur are lat: {} and long: {}'.format(jaipur_latitude, jaipur_longitude))

the coordinates of jaipur are lat: 26.916194 and long: 75.820349


In [14]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                          CLIENT_SECRET,
                                                                                                                          jaipur_latitude,
                                                                                                                          jaipur_longitude,
                                                                                                                          VERSION,
                                                                                                                          radius, LIMIT)

results = requests.get(url).json()


neighborhoods_data = results['response']['venues']# define the dataframe columns
column_names = ['Name', 'Category', 'Latitude', 'Longitude'] 

# instantiate the dataframe
jaipur_neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    neighborhood_name = data['name']
    neighborhood_lat = data['location']['lat']
    neighborhood_lon = data['location']['lng']
    if(len(data['categories'])):
        neighborhood_categories = data['categories'][0]['name']
    else:
        neighborhood_categories = None
    jaipur_neighborhoods = jaipur_neighborhoods.append({'Name': neighborhood_name,
                                          'Category': neighborhood_categories,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [15]:
address = 'Udaipur, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
udaipur_latitude = location.latitude
udaipur_longitude = location.longitude
print('the coordinates of udaipur are lat: {} and long: {}'.format(udaipur_latitude, udaipur_longitude))

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                          CLIENT_SECRET,
                                                                                                                          udaipur_latitude,
                                                                                                                          udaipur_longitude,
                                                                                                                          VERSION,
                                                                                                                          radius, LIMIT)

results = requests.get(url).json()


neighborhoods_data = results['response']['venues']# define the dataframe columns
column_names = ['Name', 'Category', 'Latitude', 'Longitude'] 

# instantiate the dataframe
udaipur_neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    neighborhood_name = data['name']
    neighborhood_lat = data['location']['lat']
    neighborhood_lon = data['location']['lng']
    if(len(data['categories'])):
        neighborhood_categories = data['categories'][0]['name']
    else:
        neighborhood_categories = None
    udaipur_neighborhoods = udaipur_neighborhoods.append({'Name': neighborhood_name,
                                          'Category': neighborhood_categories,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

the coordinates of udaipur are lat: 24.578721 and long: 73.6862571


In [16]:
print("shape of Jaipur : {} and shape of udaipur : {}".format(jaipur_neighborhoods.shape, udaipur_neighborhoods.shape))

shape of Jaipur : (110, 4) and shape of udaipur : (125, 4)


In [ ]:
def get_near_by_venues(venues_df, radius=10000):
    
    venues_list=[]
    names = venues_df["Name"]
    latitudes = venues_df["Latitude"]
    longitudes = venues_df["Longitude"]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except Exception as e:
            print(url)
            break
            continue
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Categories']
    
    return(pd.concat(venues_df, nearby_venues).drop_duplicates())
